In [ ]:
!pip install -q findspark
!pip install -q pyspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 


In [ ]:
sentenceDataFrame = spark.createDataFrame([
  (0, "Hi I am a student at aivancity"),
  (1, "aivancity is an AI school"),
  (2, "students develop key practical skills at the AI clinic")], ["id", "sentence"])

In [ ]:
sentenceDataFrame.show(truncate=False)

+---+------------------------------------------------------+
|id |sentence                                              |
+---+------------------------------------------------------+
|0  |Hi I am a student at aivancity                        |
|1  |aivancity is an AI school                             |
|2  |students develop key practical skills at the AI clinic|
+---+------------------------------------------------------+



In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [ ]:
tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.show(truncate=False)

+---+------------------------------------------------------+----------------------------------------------------------------+
|id |sentence                                              |words                                                           |
+---+------------------------------------------------------+----------------------------------------------------------------+
|0  |Hi I am a student at aivancity                        |[hi, i, am, a, student, at, aivancity]                          |
|1  |aivancity is an AI school                             |[aivancity, is, an, ai, school]                                 |
|2  |students develop key practical skills at the AI clinic|[students, develop, key, practical, skills, at, the, ai, clinic]|
+---+------------------------------------------------------+----------------------------------------------------------------+



In [ ]:
from pyspark.ml.feature import RegexTokenizer

In [ ]:
regextokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
countTokens = udf(lambda words: len(words), IntegerType())

In [ ]:
tokenized.select("sentence", "words").withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+------------------------------------------------------+----------------------------------------------------------------+------+
|sentence                                              |words                                                           |tokens|
+------------------------------------------------------+----------------------------------------------------------------+------+
|Hi I am a student at aivancity                        |[hi, i, am, a, student, at, aivancity]                          |7     |
|aivancity is an AI school                             |[aivancity, is, an, ai, school]                                 |5     |
|students develop key practical skills at the AI clinic|[students, develop, key, practical, skills, at, the, ai, clinic]|9     |
+------------------------------------------------------+----------------------------------------------------------------+------+



In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [ ]:
tokenized_out_sw = remover.transform(tokenized)
tokenized_out_sw.select("filtered").show(truncate=False)

+-------------------------------------------------------+
|filtered                                               |
+-------------------------------------------------------+
|[hi, student, aivancity]                               |
|[aivancity, ai, school]                                |
|[students, develop, key, practical, skills, ai, clinic]|
+-------------------------------------------------------+



In [ ]:
from pyspark.ml.feature import NGram  #to use sequence de mots pas des mots spérarément
ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")

ngramDataFrame = ngram.transform(tokenized_out_sw)
ngramDataFrame.select("filtered","ngrams").show(truncate=False)

+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|filtered                                               |ngrams                                                                                |
+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|[hi, student, aivancity]                               |[hi student, student aivancity]                                                       |
|[aivancity, ai, school]                                |[aivancity ai, ai school]                                                             |
|[students, develop, key, practical, skills, ai, clinic]|[students develop, develop key, key practical, practical skills, skills ai, ai clinic]|
+-------------------------------------------------------+-------------------------------------------------------------------------